
Divide a los empleados en los grupos A y B según los criterios establecidos.
#Patri
Calcula la tasa de rotación (porcentaje de empleados que dejaron la empresa) en cada grupo.
#Porcentajes segun grupo A y B
Realiza un análisis estadístico para determinar si hay una diferencia significativa en la tasa de rotación entre los grupos A y B.
#Prueba de hipotesis con método ttest_ind() con cada grupo.
H0= Afirmamos que no hay diferencia
H1= Afirmamos que si la hay, los datos no se corresponden.

Analiza los resultados.
#Afirmar si H0 o H1

Calcular la magnitud de esta relación utilizando estadísticas como la diferencia de medias por ejemplo.
#Necesitamos la media de cada grupo.
???